In [49]:
import pandas as pd
import numpy as np
import json
import geojson
from translate import Translator

In [50]:
#import climate change data
df = pd.read_csv('./raw/climate_change_orig.csv')

#delete not used columns
df.drop(['Domain Code', 'Area Code', 'Domain', 'Element Code', 'Element', 'Months Code', 'Months', 'Year Code', 'Unit', 'Flag', 'Flag Description'], axis=1, inplace=True)

#rename columns
df.columns = ['country_eng', 'year', 'value']

df.head()

,country_eng,year,value
0,Afghanistan,1961,-0.080
1,Afghanistan,1962,-0.120
2,Afghanistan,1963,0.882
3,Afghanistan,1964,-0.727
4,Afghanistan,1965,-0.200


In [51]:
#import country codes for mapping
df_country_code = pd.read_csv('./raw/country_codes.CSV', index_col=5)

#delete not used columns
df_country_code.drop(['LANG', 'LANG_NAME', 'COUNTRY_ALPHA3_CODE', 'COUNTRY_NUMERIC_CODE'], axis=1, inplace=True)

#create dict: (<countryname>: <countycode>)
country_dict = df_country_code.to_dict('dict')


In [52]:
def get_country_code(country_name, country_dict):
    """returns the country code of specific country
    
        Parameters:
        country_name: Name of the country in english (String)
        country_dict: Dictionary with country - country code mapping (Dictionary)

        Returns: country code if found, otherwise empty string (String)

       """    

    #handling countries with different names
    if (country_name == "Democratic People's Republic of Korea"):
        return "kp"
    if (country_name == "Republic of Korea"):
        return "kr"
    if (country_name == "French Southern and Antarctic Territories"):
        return "tf"
    if (country_name == "Ethiopia PDR"):
        return "et"
    if (country_name == "Midway Island"):
        return "mi"
    if (country_name == "Netherlands Antilles (former)"):
        return "an"
    if (country_name == "Pacific Islands Trust Territory"):
        return "pc"
    if (country_name == "Pitcairn Islands"):
        return "pn"
    if (country_name == "Republic of Moldova"):
        return "md"
    if (country_name == "Serbia and Montenegro"):
        return "cs"
    if (country_name == "Sudan (former)"):
        return "sd"
    if (country_name == "Svalbard and Jan Mayen Islands"):
        return "sj"
    if (country_name == "United Republic of Tanzania"):
        return "tz"
    if (country_name == "United States Virgin Islands"):
        return "vi"
    if (country_name == "Wake Island"):
        return "wk"
    if (country_name == "Wallis and Futuna Islandss"):
        return "wf"
    if (country_name == "Yugoslav SFR"):
        return "yu"
    if (country_name == "Belgium-Luxembourg"):
        return "be"
    if (country_name == "Yugoslav SFR"):
        return "yu"
    if (country_name == "Channel Islands"):
        return "ky"
    if (country_name == "China, Hong Kong SAR"):
        return "hk"
    if (country_name == "China, Macao SAR"):
        return "mo"
    if (country_name == "China, mainland"):
        return "cn"
    if (country_name == "China, Taiwan Province of"):
        return "tw"
    if (country_name == "Wallis and Futuna Islands"):
        return "wf"
    if (country_name == "Kosovo"):
        return "xk"
    if (country_name == "Namibia"):
        return "na"
    if (country_name == "Norway"):
        return "-99"
    if (country_name == "France"):
        return "fx"
    if (country_name == "Somaliland"):
        return "so"


    for key, value in country_dict.get('COUNTRY_ALPHA2_CODE').items():
        if key == country_name:
            return str(value).lower()


    return "NO_CC"

In [53]:
#mapping (country -> country code) & add to dataframe
df['country_code'] = df['country_eng'].apply(lambda x: get_country_code(x, country_dict))

In [54]:
#save as a new csv file
df.to_csv("./climate_change_cleaned.csv")

In [55]:
#open geojson default db
with open('./raw/ne_110m_admin_0_countries.geojson', 'r') as f:
    geodata = geojson.load(f)

In [56]:
def get_temperature(country_code, country_name, df):
    """returns the temperature of specific country with country code
    
        Parameters:
        country_code: ISO alpha 2 Country Code (String)
        df: dataframe with country code and temperature values (Dataframe)

        Returns: temperature value if found, otherwise 999 (Str)

       """  
    code =  country_code 

    #Handle countries with not existing country code
    if (country_code == "-99" and country_name == "France"):
            code = "fx"

    for index, row in df.iterrows():
        if row['country_code'].lower() in code.lower() and row['year'] == 2019:
            return str(row['value'])
    return "NO_DATA"

In [57]:
#add temp value to geojson properties
for feat in geodata['features']:
    feat['properties']['TEMP'] = get_temperature(feat['properties']['ISO_A2'], feat['properties']['ADMIN'], df)

In [58]:
#No climate data: kosovo, Burundi, Falkland Islands, Rwanda, Yemen

In [59]:
#Create translator instance for de, fr & it
translator_de = Translator(to_lang='de')
translator_fr = Translator(to_lang='fr')
translator_it = Translator(to_lang='it')


In [60]:
def get_translation(country_name, translator):
    """returns the the translated country
    
        Parameters:
        country_name: Country to be translated in ENGLISH
        translator: instance of translator in specific language

        Returns: translated country (Str)

       """  
    return translator.translate(country_name)

In [61]:
#add translations to geojson properties
for feat in geodata['features']:
    feat['properties']['ADMIN_DE'] = get_translation(feat['properties']['ADMIN'], translator_de)
    feat['properties']['ADMIN_FR'] = get_translation(feat['properties']['ADMIN'], translator_fr)
    feat['properties']['ADMIN_IT'] = get_translation(feat['properties']['ADMIN'], translator_it)

In [47]:
#save geojson with added temp valu and tranlsations
with open('./geojson_temp_translations.geojson', 'w') as f:
    geojson.dump(geodata, f)